# Libraries Import
## Regular Libraries

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv', index_col=0, parse_dates=['time'])
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', index_col=0, parse_dates=['time'])